In [1]:
import itertools
import sys, os

import numpy as np
import pandas as pd
from scipy.special import comb
from scipy import stats
import scipy.cluster.hierarchy as hac
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import statsmodels.api as sm

sns.set(rc={'figure.figsize':(15,8)})
sns.set_context('poster')
from sklearn.model_selection import train_test_split

In [2]:
%load_ext autoreload
%autoreload 2
import CCPA_lib as cp


In [3]:
sns.set_context('poster')
sns.set_style('white')

In [4]:
ppallete = 'RdYlGn'#'YlGn_d' #"BuGn_d"
porder = ['MED4',  'MIT9312','MIT0604', 'Natl2A', 'MIT9313']
ppallete = [ '#62A586', '#face6e','#f79934', '#7A9BC8', '#F47681', ]
# #ffd7a1
    
pedge = 'green'
apallete = "spring"
aorder = ['DE', 'DE1', '1A3', 'ATCC', 'BS11']
amarkers = ['X', 'P', 's', 'D', 'o']
aedge='orange'
epallete = ['#6cf0ba', '#d13f75', '#eeb7cb', '#b9d2d8', '#407f90']
#sns.diverging_palette(355, 217, s=74, l=50, n=4) #'Paired' # 'BrBG'# "coolwarm"
eorder = ['e1','e3', 'e4', 'e5', 'e6']
#aedge='orange'
e2_color= '#f2f2f2'
eallorder= ['e1','e2', 'e3', 'e4', 'e5', 'e6']
eallpallete = ['#6cf0ba', '#f2f2f2', '#d13f75', '#eeb7cb', '#b9d2d8', '#407f90']

efcmorder= ['e1','e2', 'e4', 'e5', ]
efcmpallete = ['#6cf0ba', '#f2f2f2',  '#eeb7cb', '#b9d2d8',]

espallete = ['#6cf0ba', '#D55282', '#538B9B']
#  604 #FCC479
#     #ffd7a1
#     #fcb14c
#     MED4 = #62A586
#     9313 #F47681
#     natl #7A9BC8

In [5]:
ppallete = 'RdYlGn'#'YlGn_d' #"BuGn_d"
porder = ['MED4',  'MIT9312','MIT0604', 'Natl2A', 'MIT9313']
ppallete = [ '#62A586', '#face6e','#f79934', '#7A9BC8', '#F47681', ]
psizes = [150,120,200,120,350]
pmarkers = ['o', 's', '^', 'D', '*']

# #ffd7a1
    
pedge = 'green'
apallete = "spring"
aorder = ['DE', 'DE1', '1A3', 'ATCC', 'BS11']
amarkers = ['P', 'X', 'h', '<','>']
apallete = ['#6dad36', '#c7e89f', '#f5c4e1', '#d5579d', '#8a0850']

aedge='orange'
epallete = ['#6cf0ba', '#d13f75', '#eeb7cb', '#b9d2d8', '#407f90']
#sns.diverging_palette(355, 217, s=74, l=50, n=4) #'Paired' # 'BrBG'# "coolwarm"
eorder = ['e1','e3', 'e4', 'e5', 'e6']
#aedge='orange'

espallete = ['#6cf0ba', '#D55282', '#538B9B']

In [6]:
fpath = 'anat'
xlsx_fnames = os.listdir(fpath)


In [7]:
xlsx_dfs = pd.read_excel(os.path.join(fpath, xlsx_fnames[0]), sheet_name=None, index_col=0)

In [25]:
def clean_df(d, fname, sheetname):
    df = d.T
    df = df.replace(to_replace=r'^n=[0-9]+', value=np.nan, regex=True)
    df = df.dropna(how='all', axis=1)
    df = df.dropna(how='all', axis=0)
    df = df.reset_index(drop=True)
    experiments = df.columns.str.replace(' day', '').str.strip()
    experiments = experiments.str.replace('_DAY', '').str.strip()
    
    #colnames = 
    colnames = (pd.Series(df.columns.str.endswith(' day')) | pd.Series(df.columns.str.endswith('_DAY'))).map({True: 'day', False: 'FL'})
    df.columns = pd.MultiIndex.from_frame(pd.DataFrame({'experiment': experiments, 'colname': colnames}))
    df = df.stack(level='experiment').reset_index().drop(columns=['level_0']).reset_index(drop=True)
    df['excelfile'] = fname
    df['sheet'] = sheetname
    return df


In [34]:
dfs = []
for fname in xlsx_fnames:
    xlsx_dfs = pd.read_excel(os.path.join(fpath, fname), sheet_name=None, index_col=0)
    dfs.extend([clean_df(d, fname, sheet) for sheet, d in xlsx_dfs.items()])

In [35]:
df = pd.concat(dfs)

In [36]:
df.isna().sum()

colname
experiment     0
FL            16
day            0
excelfile      0
sheet          0
dtype: int64

In [37]:
df.loc[df.FL.isna()]

colname,experiment,FL,day,excelfile,sheet
5,MED4_10UE_17C 31.5,NaN,0.000000,MED4 סיכום עקומות גדילה.xlsx,MED4_10UE_17C
5,MED4_50UE_17C 31.5,NaN,0.000000,MED4 סיכום עקומות גדילה.xlsx,MED4_50UE_17C
4,9312_10UE_17C 31.5,NaN,0.000000,MIT9312 סיכום עקומות גדילה.xlsx,9312_10UE_17C
3,9312_50UE_17C 31.5,NaN,0.000000,MIT9312 סיכום עקומות גדילה.xlsx,9312_50UE_17C
43,MIT9312_10UE_20C_1A3 8.6,NaN,3.101389,MIT9312 סיכום עקומות גדילה.xlsx,9312_10UE_20C_1A3
4,MIT9313_10UE_17C 31.5,NaN,0.000000,MIT9313 סיכום עקומות גדילה.xlsx,9313_10UE_17C
43,MIT9313_10UE_20C_1A3 8.6,NaN,3.101389,MIT9313 סיכום עקומות גדילה.xlsx,9313_10UE_20C_1A3
4,NATL2A_10UE_17C 31.5,NaN,0.000000,NATL סיכום עקומות גדילה.xlsx,NATL_10UE_17C
39,NATL2A_10UE_20C 8.6,NaN,3.097222,NATL סיכום עקומות גדילה.xlsx,NATL_10UE_20C
39,NATL2A_50UE_20C 8.6,NaN,3.098611,NATL סיכום עקומות גדילה.xlsx,NATL_50UE_20C


In [38]:
def calculate_lag(d):
    minday_fl_threshold = 0.1
    r2_threshold = 0.9
    reg = None 
    intercept = None
    coef = None
    r2 = -1000 
    L = None
    maxday = None
    maxFL = None

    d = d.reset_index(drop=True)
    if d.shape[0] > 2:
        maxindex = d.FL.idxmax()
        maxday = d.loc[maxindex]['day']
        maxFL = d.loc[maxindex]['FL']
        minday = d.loc[(d.day < maxday) & (d.FL <= minday_fl_threshold), 'day'].max()
        if minday is np.NaN:
            minday = d.day.min()

        minday = min(maxday-7,minday)
        dfit = d.loc[(d.day > minday) & (d.day <= maxday)]
        if dfit.shape[0] > 2:
            reg, intercept, coef, r2 = cp.fit_regression(dfit, 'day', 'logFL')

            L = maxday - ((d.loc[maxindex]['logFL'] - d.loc[0]['logFL']) / coef)
    return pd.Series({
         'maxday' : maxday,
#         'minday' : minday,
         'maxFL' : maxFL,
#         'logFL0' : d.loc[0]['logFL'],
#         'log10FL0' : np.log10(d.loc[0]['FL']),
        'intercept': intercept,
        'Growth Rate': coef,
        'r2': r2,
        'Lag' : L,
    })

def get_prev_max_day_df(d, minmaxday_threshold):
    d = d.reset_index(drop=True)
    if d.shape[0] < 3:
        return None
    
    maxindex = d.FL.idxmax()
    maxday = d.loc[maxindex]['day']
    maxdaysdf = d.loc[(d.FL > d.FL.shift()) & (d.FL > d.FL.shift(-1)) & 
                    (d.day < maxday)].reset_index(drop=True)
    if maxdaysdf.shape[0] > 0:
        newmaxindex = maxdaysdf.FL.idxmax()
        newmaxday = maxdaysdf.loc[newmaxindex]['day']

        if ((newmaxday is not np.NaN) and #(newmaxday >= minmaxday_threshold) and 
            (d.loc[d.day == newmaxday, 'FL'].max() > (d.FL.max() / 2))):
            df_prevmax = d.loc[d.day < newmaxday]
            return df_prevmax
    return None

def get_no_outlier_df(d, minday_fl_threshold):
    d_no_outlier = d.loc[(d.FL > minday_fl_threshold) | 
                         (d.FL.shift() < minday_fl_threshold*2) |
                         (d.FL.shift(-1) < minday_fl_threshold*2)
                        ]
    return d_no_outlier

def get_without_max_df(d):
    d = d.reset_index(drop=True)
    maxindex = d.FL.idxmax()
    maxday = d.loc[maxindex]['day']
    d_withoutmaxday = d.loc[d.day < maxday]
    return d_withoutmaxday

def get_df_list(d, minday_fl_threshold, minmaxday_threshold):
    df_list = [d]
    try:
        dnomax = get_without_max_df(d)
        df_list.append(dnomax)
        dnomax2 = get_without_max_df(dnomax)
        df_list.append(dnomax2)
        df_list1 = df_list.copy()
        for x in df_list1:
            df_list.append(get_no_outlier_df(x, minday_fl_threshold))
    except Exception:
        pass

    return df_list

def calculate_lag_wrap(d):
    minday_fl_threshold = 0.07
    minmaxday_threshold = 4
    df_list = get_df_list(d, minday_fl_threshold, minmaxday_threshold)
    d2 = get_prev_max_day_df(d,minmaxday_threshold)
    if d2 is not None:
        df_list.extend(get_df_list(d2, minday_fl_threshold, minmaxday_threshold))
    reslist = [calculate_lag(x) for x in df_list]

    sorted_reslist = sorted(reslist, key=lambda x : x['r2'], reverse=True)
    print('.', end='')
    return sorted_reslist[0]           

In [39]:
def computeL(maxday, maxlogcells, logcells0, coef):
    L = maxday - ((maxlogcells - logcells0) / coef)
    return L


In [40]:
def computeMaxDay(maxlogcells, logcells0, coef, L):
    maxday = ((maxlogcells - logcells0) / coef) + L
    return maxday


In [41]:
df.columns

Index(['experiment', 'FL', 'day', 'excelfile', 'sheet'], dtype='object', name='colname')

In [42]:
df.dropna(axis=0, subset=['FL'], inplace=True)

In [43]:
df['logFL'] = np.log(df['FL'])

C:\Users\wosnat\Anaconda3\lib\site-packages\pandas\core\series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [44]:
df.isna().sum()

colname
experiment    0
FL            0
day           0
excelfile     0
sheet         0
logFL         0
dtype: int64

In [45]:
df.loc[df.day.isna()]

colname,experiment,FL,day,excelfile,sheet,logFL


In [46]:
lagsdf = df.groupby([ 'excelfile', 'sheet', 'experiment']).apply(calculate_lag_wrap)\
        .reset_index()

...........................................................................................................................................................................................................................................................................................................................................................................................

In [47]:
lagsdf.head()

,excelfile,sheet,experiment,maxday,maxFL,intercept,Growth Rate,r2,Lag
0,MED4 סיכום עקומות גדילה.xlsx,MED4_100UE_17C,MED4_100UE_17C 13.7,5.872222,2.203,-1.784376,0.448097,0.994012,-0.008153
1,MED4 סיכום עקומות גדילה.xlsx,MED4_100UE_17C,MED4_100UE_17C 19.7,3.113889,1.101,-1.591101,0.558810,0.992691,0.034540
2,MED4 סיכום עקומות גדילה.xlsx,MED4_100UE_17C,MED4_100UE_17C 21.6,2.907639,0.833,-1.855727,0.578663,0.944086,-0.199141
3,MED4 סיכום עקומות גדילה.xlsx,MED4_100UE_17C,MED4_100UE_17C 28.6,3.043750,1.045,-2.077794,0.717871,0.994077,0.040451
4,MED4 סיכום עקומות גדילה.xlsx,MED4_100UE_20C,MED4_100UE_20C 10.5,6.256944,1.516,-1.460676,0.343780,0.918491,0.465122


In [48]:
lagsdf.to_excel('anat_growth_rates.xlsx')

In [49]:
lagsdf.describe()

,maxday,maxFL,intercept,Growth Rate,r2,Lag
count,379.000000,379.000000,367.000000,367.000000,379.000000,367.000000
mean,7.005207,2.462773,-1.426496,0.354199,-30.722508,-inf
std,4.130141,1.937544,1.083278,0.181911,175.500940,NaN
min,0.000000,0.149000,-6.344043,0.029658,-1000.000000,-inf
25%,3.703472,0.914500,-2.142174,0.232353,0.966555,-0.077486
50%,6.820833,1.911000,-1.455617,0.328204,0.987135,0.040451
75%,9.127778,3.679500,-0.672281,0.441964,0.995229,0.432859
max,21.962500,9.072000,1.387176,1.043177,1.000000,10.228003


In [50]:
lagsdf.r2.value_counts()

-1000.000000    12
 0.994948        2
 0.992691        2
 0.947788        1
 0.991120        1
                ..
 0.994306        1
 0.977541        1
 0.995851        1
 0.985112        1
 0.963825        1
Name: r2, Length: 366, dtype: int64

In [51]:
lagsdf.groupby('excelfile')['r2'].value_counts()

excelfile                                r2          
MED4 סיכום עקומות גדילה.xlsx             -1000.000000    1
                                          0.865814       1
                                          0.918491       1
                                          0.944086       1
                                          0.957135       1
                                                        ..
קצב גדילה ניסויי זמנים (version 1).xlsx   0.993473       1
                                          0.995640       1
                                          0.999253       1
                                          0.999829       1
                                          0.999993       1
Name: r2, Length: 372, dtype: int64